## GRACE/GRACE-FO Harmonic Visualization Program

This notebook uses standard Python tools to demonstrate more visualization of the Gravity Recovery and Climate Experiment (GRACE) and the GRACE Follow-On (GRACE-FO) Level-2 spherical harmonic products.

This notebook uses Jupyter widgets to set parameters for visualizing the GRACE/GRACE-FO Level-2 products as spherical harmonics.
The widgets can be installed as described below.  
```bash
pip3 install --user ipywidgets
jupyter nbextension enable --py --user widgetsnbextension
jupyter-notebook
```

### Load necessary modules for running the notebook

In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 40
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
import ipywidgets as widgets
from IPython.display import HTML, Latex

import gravity_toolkit.utilities
from gravity_toolkit.grace_find_months import grace_find_months
from gravity_toolkit.grace_input_months import grace_input_months
from gravity_toolkit.read_GIA_model import read_GIA_model
from gravity_toolkit.harmonics import harmonics
from gravity_toolkit.read_love_numbers import read_love_numbers
from gravity_toolkit.units import units
from gravity_toolkit.gauss_weights import gauss_weights
from gravity_toolkit.ocean_stokes import ocean_stokes

### Set the GRACE/GRACE-FO Data Directory
Directory should contain:  
- Technical notes with SLR coefficients  
- Subdirectory with geocenter coefficients  
- Subdirectories for each processing center  

In [ ]:
# set the directory with GRACE/GRACE-FO data
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)
# update local data with PO.DAAC https servers
updateCheckbox = widgets.Checkbox(
    value=True,
    description='Update data?',
    disabled=False
)
widgets.VBox([dirText,updateCheckbox])

### Update Data in Directory

In [ ]:
# extract directory value from widget
base_dir = os.path.expanduser(dirText.value)
# if updating the local data
if updateCheckbox.value:
    # run podaac sync program to get latest data
    !podaac_grace_sync.py --directory=$base_dir
    # run GRACE date program to verify months
    !run_grace_date.py --directory=$base_dir --verbose
    # get geocenter data from Sutterley and Velicogna (2019)
    gravity_toolkit.utilities.from_figshare(base_dir)

### Set GRACE/GRACE-FO Parameters
These parameters describe the specific GRACE/GRACE-FO product and the months of data to read  

- GRACE/GRACE-FO Processing Center
    * CSR: University of Texas Center for Space Research  
    * GFZ: German Research Centre for Geosciences (GeoForschungsZentrum)
    * JPL: Jet Propulsion Laboratory    
    * CNES: French Centre National D'Etudes Spatiales
- GRACE/GRACE-FO Data Release
- GRACE/GRACE-FO Data Product
    * GAA: non-tidal atmospheric correction  
    * GAB: non-tidal oceanic correction  
    * GAC: combined non-tidal atmospheric and oceanic correction  
    * GAD: GRACE/GRACE-FO ocean bottom pressure product  
    * GSM: corrected monthly GRACE/GRACE-FO static field product
- GRACE/GRACE-FO Date Range

In [ ]:
# dropdown menu for setting processing center
# CSR: University of Texas Center for Space Research  
# GFZ: German Research Centre for Geosciences (GeoForschungsZentrum)
# JPL: Jet Propulsion Laboratory    
# CNES: French Centre National D'Etudes Spatiales
proc_list = ['CSR', 'GFZ', 'JPL', 'CNES']
proc_default = 'CSR'
procDropdown = widgets.Dropdown(
    options=proc_list,
    value=proc_default,
    description='Center:',
    disabled=False,
)

# dropdown menu for setting data release
drel_list = ['RL04', 'RL05', 'RL06']
drel_default = 'RL06'
drelDropdown = widgets.Dropdown(
    description='Release:',
    options=drel_list,
    value=drel_default,
    disabled=False,
)

# dropdown menu for setting data product
# GAA: non-tidal atmospheric correction  
# GAB: non-tidal oceanic correction  
# GAC: combined non-tidal atmospheric and oceanic correction  
# GAD: GRACE/GRACE-FO ocean bottom pressure product  
# GSM: corrected monthly GRACE/GRACE-FO static field product
dset_list = ['GAC', 'GAD', 'GSM']
dset_default = 'GSM'
dsetDropdown = widgets.Dropdown(
    description='Product:',
    options=dset_list,
    value=dset_default,    
    disabled=False,
)
    
# find available months for data product
total_months = grace_find_months(base_dir, procDropdown.value,
    drelDropdown.value, DSET=dsetDropdown.value)
# select months to run
# https://tsutterley.github.io/data/GRACE-Months.html
options=['{0:03d}'.format(m) for m in total_months['months']]
monthsSelect = widgets.SelectMultiple(
    options=options,
    value=options,
    description='Months:',
    disabled=False
)

# function for setting the data release
def set_release(sender):
    if (procDropdown.value == 'CNES'):
        drel_list = ['RL03', 'RL04', 'RL05']
        drel_default = 'RL05'
    else:
        drel_list = ['RL04', 'RL05', 'RL06']
        drel_default = 'RL06'
    drelDropdown.options=drel_list
    drelDropdown.value=drel_default

# function for setting the data product
def set_product(sender):
    if ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL01')):
        dset_list = ['GAC', 'GSM']
    elif ((procDropdown.value == 'CNES') and drelDropdown.value in ('RL02','RL03','RL05')):
        dset_list = ['GAA', 'GAB', 'GSM']
    elif ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL04')):
        dset_list = ['GSM']
    elif (procDropdown.value == 'CSR'):
        dset_list = ['GAC', 'GAD', 'GSM']
    else:
        dset_list = ['GAA', 'GAB', 'GAC', 'GAD', 'GSM']
    dsetDropdown.options=dset_list
    dsetDropdown.value=dset_default
    
# function for updating the available month
def update_months(sender):
    total_months = grace_find_months(base_dir, procDropdown.value,
        drelDropdown.value, DSET=dsetDropdown.value)
    options=['{0:03d}'.format(m) for m in total_months['months']]
    monthsSelect.options=options
    monthsSelect.value=options

# watch widgets for changes
procDropdown.observe(set_release)
drelDropdown.observe(set_product)
procDropdown.observe(set_product)
procDropdown.observe(update_months)
drelDropdown.observe(update_months)

# display widgets for setting GRACE/GRACE-FO parameters
widgets.VBox([procDropdown,drelDropdown,dsetDropdown,monthsSelect])

### Set Parameters for Reading GRACE/GRACE-FO Data
These parameters describe processing steps and corrections to be applied when reading the GRACE/GRACE-FO data

- Maximum Degree and Order
- Geocenter product (Degree 1)
- Oblateness product (<i>C</i><sub>20</sub>)
- Figure axis product (<i>C</i><sub>21</sub> and <i>S</i><sub>21</sub>)
- Azimuthal dependence product (<i>C</i><sub>22</sub> and <i>S</i><sub>22</sub>)
- Low Degree Zonal products (<i>C</i><sub>30</sub> and <i>C</i><sub>50</sub>)
- Pole Tide Correction from [Wahr et al. (2015)](https://doi.org/10.1002/2015JB011986)  
- Atmospheric Correction as described in [Fagiolini et al. (2015)](https://doi.org/10.1093/gji/ggv276)  

In [ ]:
# set the spherical harmonic truncation parameters
lmax_default = {}
lmax_default['CNES'] = {'RL01':50,'RL02':50,'RL03':80}
# CSR RL04/5/6 at LMAX 60
lmax_default['CSR'] = {'RL04':60,'RL05':60,'RL06':60}
# GFZ RL04/5 at LMAX 90
# GFZ RL06 at LMAX 60
lmax_default['GFZ'] = {'RL04':90,'RL05':90,'RL06':60}
# JPL RL04/5/6 at LMAX 60
lmax_default['JPL'] = {'RL04':60,'RL05':60,'RL06':60}
# text entry for spherical harmonic degree
lmaxText = widgets.BoundedIntText(
    min=0,
    max=lmax_default[procDropdown.value][drelDropdown.value],
    value=lmax_default[procDropdown.value][drelDropdown.value],
    step=1,
    description='<i>&#8467;</i><sub>max</sub>:',
    disabled=False
)

# text entry for spherical harmonic order
mmaxText = widgets.BoundedIntText(
    min=0,
    max=lmax_default[procDropdown.value][drelDropdown.value],
    value=lmax_default[procDropdown.value][drelDropdown.value],
    step=1,    
    description='<i>m</i><sub>max</sub>:',
    disabled=False
)

# dropdown menu for setting geocenter
# Tellus: GRACE/GRACE-FO TN-13 from PO.DAAC
#    https://grace.jpl.nasa.gov/data/get-data/geocenter/
# SLR: satellite laser ranging from CSR
#    ftp://ftp.csr.utexas.edu/pub/slr/geocenter/
# SLF: Sutterley and Velicogna, Remote Sensing (2019)
#    https://www.mdpi.com/2072-4292/11/18/2108
geocenter_list = ['[none]', 'Tellus', 'SLR', 'SLF']
geocenter_default = 'SLF' if (dsetDropdown.value == 'GSM') else '[none]'
geocenterDropdown = widgets.Dropdown(
    options=geocenter_list,
    value=geocenter_default,
    description='Geocenter:',
    disabled=False,
)

# SLR C20
C20_list = ['[none]','CSR','GSFC']
C20_default = 'GSFC' if (dsetDropdown.value == 'GSM') else '[none]'
C20Dropdown = widgets.Dropdown(
    options=C20_list,
    value=C20_default,
    description='SLR C20:',
    disabled=False,
)

# SLR C21 and S21
C21_list = ['[none]','CSR']
C21Dropdown = widgets.Dropdown(
    options=C21_list,
    value='[none]',
    description='SLR C21:',
    disabled=False,
)

# SLR C22 and S22
C22_list = ['[none]','CSR']
C22Dropdown = widgets.Dropdown(
    options=C22_list,
    value='[none]',
    description='SLR C22:',
    disabled=False,
)

# SLR C30
C30_list = ['[none]','CSR','GSFC']
C30_default = 'GSFC' if (dsetDropdown.value == 'GSM') else '[none]'
C30Dropdown = widgets.Dropdown(
    options=C30_list,
    value=C30_default,
    description='SLR C30:',
    disabled=False,
)

# SLR C50
C50_list = ['[none]','CSR','GSFC']
C50Dropdown = widgets.Dropdown(
    options=C50_list,
    value='[none]',
    description='SLR C50:',
    disabled=False,
)

# Pole Tide Drift (Wahr et al., 2015) for Release-5
poletide_default = True if ((drelDropdown.value == 'RL05')
    and (dsetDropdown.value == 'GSM')) else False
poletideCheckbox = widgets.Checkbox(
    value=poletide_default,
    description='Pole Tide Corrections',
    disabled=False
)


# ECMWF Atmospheric Jump Corrections for Release-5
atm_default = True if (dsetDropdown.value == 'RL05') else False
atmCheckbox = widgets.Checkbox(
    value=atm_default,
    description='ATM Corrections',
    disabled=False
)

# functions for setting the spherical harmonic truncation
def set_SHdegree(sender):
    lmaxText.max=lmax_default[procDropdown.value][drelDropdown.value]
    lmaxText.value=lmax_default[procDropdown.value][drelDropdown.value]

def set_SHorder(sender):
    mmaxText.max=lmaxText.value
    mmaxText.value=lmaxText.value

# functions for setting pole tide and atmospheric corrections for Release-5
def set_pole_tide(sender):
    poletideCheckbox.value = True if ((drelDropdown.value == 'RL05')
        and (dsetDropdown.value == 'GSM')) else False

def set_atm_corr(sender):
    atmCheckbox.value = True if (drelDropdown.value == 'RL05') else False

# watch processing center widget for changes
procDropdown.observe(set_SHdegree)
# watch data release widget for changes
drelDropdown.observe(set_SHdegree)
drelDropdown.observe(set_pole_tide)
drelDropdown.observe(set_atm_corr)
# watch data product widget for changes
dsetDropdown.observe(set_pole_tide)
# watch spherical harmonic degree widget for changes
lmaxText.observe(set_SHorder)
        
# display widgets for setting GRACE/GRACE-FO read parameters
widgets.VBox([lmaxText,mmaxText,geocenterDropdown,
    C20Dropdown,C21Dropdown,C22Dropdown,C30Dropdown,C50Dropdown,
    poletideCheckbox,atmCheckbox])

### Read GRACE/GRACE-FO data
This step extracts the parameters chosen above and then reads the GRACE/GRACE-FO data applying the specified procedures  

In [ ]:
# extract values from widgets
PROC = procDropdown.value
DREL = drelDropdown.value
DSET = dsetDropdown.value
months = [int(m) for m in monthsSelect.value]
LMAX = lmaxText.value
MMAX = mmaxText.value
DEG1 = geocenterDropdown.value
SLR_C20 = C20Dropdown.value
SLR_21 = C21Dropdown.value
SLR_22 = C22Dropdown.value
SLR_C30 = C30Dropdown.value
SLR_C50 = C50Dropdown.value
POLE_TIDE = poletideCheckbox.value
ATM = atmCheckbox.value

# read GRACE/GRACE-FO data for parameters
start_mon = np.min(months)
end_mon = np.max(months)
missing = sorted(set(np.arange(start_mon,end_mon+1)) - set(months))
Ylms = grace_input_months(base_dir, PROC, DREL, DSET,
    LMAX, start_mon, end_mon, missing, SLR_C20, DEG1, MMAX=MMAX,
    SLR_21=SLR_21, SLR_22=SLR_22, SLR_C30=SLR_C30, SLR_C50=SLR_C50,
    POLE_TIDE=POLE_TIDE, ATM=ATM)
# create harmonics object and remove mean
GRACE_Ylms = harmonics().from_dict(Ylms)
GRACE_Ylms.mean(apply=True)
# number of time steps
nt = len(months)

### Set Parameters to Visualize Harmonics
These parameters specify corrections and filtering steps

- GIA Correction  
- Remove Specific Harmonic Fields  
- Redistribute Removed Fields over the Ocean  
- Gaussian Smoothing Radius in kilometers  
- Filter (destripe) harmonics [(Swenson and Wahr, 2006)](https://doi.org/10.1029/2005GL025285)  
- Output spherical harmonic units  
    1) equivalent water thickness (cm)  
    2) geoid height (mm)  

In [ ]:
# set the GIA file
# files come in different formats depending on the group
giaText = widgets.Text(
    value='',
    description='GIA File:',
    disabled=False
)

# dropdown menu for setting GIA model
# IJ05-R2: Ivins R2 GIA Models
# W12a: Whitehouse GIA Models
# SM09: Simpson/Milne GIA Models
# ICE6G: ICE-6G GIA Models
# Wu10: Wu (2010) GIA Correction
# AW13-ICE6G: Geruo A ICE-6G GIA Models
# Caron: Caron JPL GIA Assimilation
# ICE6G-D: ICE-6G Version-D GIA Models
# ascii: GIA reformatted to ascii
# netCDF4: GIA reformatted to netCDF4
# HDF5: GIA reformatted to HDF5
gia_list = ['[None]','IJ05-R2','W12a','SM09','ICE6G',
    'Wu10','AW13-ICE6G','Caron','ICE6G-D',
    'ascii','netCDF4','HDF5']
gia_default = '[None]'
giaDropdown = widgets.Dropdown(
    options=gia_list,
    value=gia_default,
    description='GIA Type:',
    disabled=False,
)

# set the files to be removed
removeText = widgets.Text(
    value='',
    description='Rem. Files:',
    disabled=False
)

# dropdown menu for setting remove file type
# netCDF4: single netCDF4 file
# HDF5: single HDF5 file
# index (ascii): index of monthly ascii files
# index (netCDF4): index of monthly netCDF4 files
# index (HDF5): index of monthly HDF5 files
remove_list = ['[None]','netCDF4','HDF5',
    'index (ascii)','index (netCDF4)','index (HDF5)']
remove_default = '[None]'
removeDropdown = widgets.Dropdown(
    options=remove_list,
    value=remove_default,
    description='Rem. Type:',
    disabled=False,
)

# redestribute removed file mass over the ocean 
redistributeCheckbox = widgets.Checkbox(
    value=False,
    description='Redistribute Removed',
    disabled=False
)

# path to land-sea mask for ocean redistribuion
landmaskText = widgets.Text(
    value='',
    description='Mask File:',
    disabled=False
)

# text entry for Gaussian Smoothing Radius in km
gaussianText = widgets.BoundedFloatText(
    value=300,
    min=0,
    max=1000.0,
    step=50,
    description='Gaussian:',
    disabled=False
)

# Destripe Spherical Harmonics
destripeCheckbox = widgets.Checkbox(
    value=True,
    description='Destripe',
    disabled=False
)

# dropdown menu for setting units
# 1: cm of water thickness
# 2: mm of geoid height
unit_list = ['cmwe', 'mmGH']
unit_label = ['cm', 'mm']
unit_name = ['Equivalent Water Thickness','Geoid Height']
unit_default = 'cmwe'
unitsDropdown = widgets.Dropdown(
    options=unit_list,
    value=unit_default,
    description='Units:',
    disabled=False,
)

# display widgets for setting GRACE/GRACE-FO read parameters
widgets.VBox([giaText,giaDropdown,removeText,removeDropdown,
    redistributeCheckbox,landmaskText,gaussianText,
    destripeCheckbox,unitsDropdown])

### Convert GRACE/GRACE-FO harmonics to output units
This step extracts the parameters chosen above and then converts the GRACE/GRACE-FO harmonics to the specified units applying the specified corrections and filtering procedures

- Read GIA model for correcting GRACE/GRACE-FO data  
- Read harmonics to be removed from the GRACE/GRACE-FO data  
- Calculate coefficients for converting to the output units   

In [ ]:
# read load love numbers file
# PREM outputs from Han and Wahr (1995)
# https://doi.org/10.1111/j.1365-246X.1995.tb01819.x
love_numbers_file = gravity_toolkit.utilities.get_data_path(['data','love_numbers'])
header = 2
columns = ['l','hl','kl','ll']
# LMAX of load love numbers from Han and Wahr (1995) is 696.
# from Wahr (2007) linearly interpolating kl works
# however, as we are linearly extrapolating out, do not make
# LMAX too much larger than 696
# read arrays of kl, hl, and ll Love Numbers
hl,kl,ll = read_love_numbers(love_numbers_file, LMAX=LMAX, HEADER=header,
    COLUMNS=columns, REFERENCE='CF', FORMAT='tuple')

# read GIA data
GIA = giaDropdown.value
GIA_Ylms_rate = read_GIA_model(giaText.value, GIA=GIA, LMAX=LMAX, MMAX=MMAX)
gia_str = '' if (GIA == '[None]') else '_{0}'.format(GIA_Ylms_rate['title'])
# calculate the monthly mass change from GIA
GIA_Ylms = GRACE_Ylms.zeros_like()
GIA_Ylms.time[:] = np.copy(GRACE_Ylms.time)
GIA_Ylms.month[:] = np.copy(GRACE_Ylms.month)
# monthly GIA calculated by gia_rate*time elapsed
# finding change in GIA each month
for t in range(nt):
    GIA_Ylms.clm[:,:,t] = GIA_Ylms_rate['clm']*(GIA_Ylms.time[t]-2003.3)
    GIA_Ylms.slm[:,:,t] = GIA_Ylms_rate['slm']*(GIA_Ylms.time[t]-2003.3)

# if redistributing removed mass over the ocean
if redistributeCheckbox.value:
    # read Land-Sea Mask and convert to spherical harmonics
    LSMASK = os.path.expanduser(landmaskText)
    ocean_Ylms = ocean_stokes(LSMASK,LMAX,MMAX=MMAX,LOVE=(hl,kl,ll))
    
# read data to be removed from GRACE/GRACE-FO monthly harmonics
remove_Ylms = GRACE_Ylms.zeros_like()
remove_Ylms.time[:] = np.copy(GRACE_Ylms.time)
remove_Ylms.month[:] = np.copy(GRACE_Ylms.time)
# If there are files to be removed from the GRACE/GRACE-FO data
remove_files = removeText.value.split(',') if removeText.value else []
# for each file separated by commas
for f in remove_files:
    if (removeDropdown.value == 'netCDF4'):
        # read netCDF4 file
        Ylms = harmonics().from_netCDF4(f)
    elif (removeDropdown.value == 'HDF5'):
        # read HDF5 file
        Ylms = harmonics().from_HDF5(f)
    elif (removeDropdown.value == 'index (ascii)'):
        # read index of ascii files
        Ylms = harmonics().from_index(f,'ascii')
    elif (removeDropdown.value == 'index (netCDF4)'):
        # read index of netCDF4 files
        Ylms = harmonics().from_index(f,'netCDF4')
    elif (removeDropdown.value == 'index (HDF5)'):
        # read index of HDF5 files
        Ylms = harmonics().from_index(f,'HDF5')
    # reduce to months of interest and truncate to range
    Ylms = Ylms.subset(months).truncate(LMAX,mmax=MMAX)
    # redistribute removed mass over the ocean
    if redistributeCheckbox.value:
        # calculate ratio between total removed mass and
        # a uniformly distributed cm of water over the ocean
        ratio = Ylms.clm[0,0,:]/ocean_Ylms.clm[0,0]
        # for each spherical harmonic
        for m in range(0,MMAX+1):
            for l in range(m,LMAX+1):
                # remove the ratio*ocean Ylms from Ylms
                Ylms.clm[l,m,:]-=ratio*ocean_Ylms.clm[l,m]
                Ylms.slm[l,m,:]-=ratio*ocean_Ylms.slm[l,m]
    # add the harmonics to be removed to the total
    remove_Ylms.add(Ylms)

# gaussian smoothing radius in km (Jekeli, 1981)
RAD = gaussianText.value
if (RAD != 0):
    wt = 2.0*np.pi*gauss_weights(RAD,LMAX)
    gw_str = '_r{0:0.0f}km'.format(RAD)
else:
    # else = 1
    wt = np.ones((LMAX+1))
    gw_str = ''

# destriping the GRACE/GRACE-FO harmonics
ds_str = '_FL' if destripeCheckbox.value else ''

# Setting units factor for output
UNITS = unitsDropdown.index + 1
# dfactor is the degree dependent coefficients
# for specific spherical harmonic output units
if (UNITS == 1):
    # 1: cmwe, centimeters water equivalent
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).cmwe
elif (UNITS == 2):
    # 2: mmGH, millimeters geoid height
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mmGH

# converting harmonics to truncated, smoothed coefficients in units
if destripeCheckbox.value:
    Ylms = GRACE_Ylms.destripe()
    Ylms.subtract(remove_Ylms.destripe())
else:
    Ylms = GRACE_Ylms.copy()
    Ylms.subtract(remove_Ylms)
# Remove GIA estimate for month
Ylms.subtract(GIA_Ylms)
# smooth harmonics and convert to output units
Ylms.convolve(dfactor*wt)
# create merged masked array
triangle = Ylms.to_masked_array()

### Set parameters for creating animation
This step specifies the [colormap](https://matplotlib.org/gallery/color/colormap_reference.html), the normalization for the contour plot colors and the step for the color bar.


In [ ]:
# all listed colormaps in matplotlib version
cmap_set = set(plt.cm.datad.keys()) | set(plt.cm.cmaps_listed.keys())
# colormaps available in this program
# (no reversed, qualitative or miscellaneous)
cmaps = {}
cmaps['Perceptually Uniform Sequential'] = ['viridis',
    'plasma','inferno','magma','cividis']
cmaps['Sequential'] = ['Greys','Purples','Blues','Greens',
    'Oranges','Reds','YlOrBr','YlOrRd','OrRd','PuRd','RdPu',
    'BuPu','GnBu','PuBu','YlGnBu','PuBuGn','BuGn','YlGn']
cmaps['Sequential (2)'] = ['binary','gist_yarg','gist_gray', 
    'gray','bone','pink','spring','summer','autumn','winter',
    'cool','Wistia','hot','afmhot','gist_heat','copper']
cmaps['Diverging'] = ['PiYG','PRGn','BrBG','PuOr','RdGy','RdBu',
    'RdYlBu','RdYlGn','Spectral','coolwarm', 'bwr','seismic']
cmaps['Cyclic'] = ['twilight','twilight_shifted','hsv']
# create list of available colormaps in program
cmap_list = []
for key,val in cmaps.items():
    cmap_list.extend(val)
# reduce colormaps to available in program and matplotlib
cmap_set &= set(cmap_list)
# dropdown menu for setting colormap
cmapDropdown = widgets.Dropdown(
    options=sorted(cmap_set),
    value='viridis',
    description='Colormap:',
    disabled=False,
)

# Reverse the colormap
cmapCheckbox = widgets.Checkbox(
    value=False,
    description='Reverse Colormap',
    disabled=False
)

# display widgets for setting GRACE/GRACE-FO plot parameters
widgets.VBox([cmapDropdown,cmapCheckbox])

### Create animation of GRACE/GRACE-FO months

In [ ]:
# plot spherical harmonics for each month
fig, ax1 = plt.subplots(num=1,figsize=(8,4))

# levels and normalization for plot range
cmap_reverse_flag = '_r' if cmapCheckbox.value else ''
cmap = plt.cm.get_cmap(cmapDropdown.value + cmap_reverse_flag)
cmap.set_bad('lightgrey',1.)
# imshow = show image (interpolation nearest for blocks)
im = ax1.imshow(np.ma.zeros((LMAX+1,LMAX+1)), interpolation='nearest',
    cmap=cmap, extent=(-LMAX,LMAX,LMAX,0), animated=True)
#-- Z color limit between -1 and 1
im.set_clim(-1.0,1.0)

# add date label (year-calendar month e.g. 2002-01)
time_text = ax1.text(0.025, 0.025, '', transform=fig.transFigure,
    color='k', size=24, ha='left', va='baseline')

# add text to label Slm side and Clm side
t1=ax1.text(0.40,0.93,'$S_{lm}$', size=24, transform=ax1.transAxes,
    ha="center", va="center")
t2=ax1.text(0.60,0.93,'$C_{lm}$', size=24, transform=ax1.transAxes,
    ha="center", va="center")
# add x and y labels
ax1.set_ylabel('Degree (l)', fontsize=13)
ax1.set_xlabel('Order (m)', fontsize=13)
for tick in ax1.xaxis.get_major_ticks():
    tick.label.set_fontsize(13)
for tick in ax1.yaxis.get_major_ticks():
    tick.label.set_fontsize(13)

# Add horizontal colorbar and adjust size
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# pad = distance from main plot axis
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax1, extend='both', extendfrac=0.0375,
    orientation='vertical', pad=0.025, shrink=0.85,
    aspect=15, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_ylabel(unit_name[unitsDropdown.index],
    labelpad=5, fontsize=13)
cbar.ax.set_xlabel(unit_label[unitsDropdown.index],
    fontsize=13, rotation=0)
cbar.ax.xaxis.set_label_coords(0.5,1.065)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=14, labelsize=13,
    direction='in')
    
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax1.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.075,right=0.99,bottom=0.07,top=0.99)

# animate frames
def animate_frames(i):
    # set image
    im.set_data(triangle[:,:,i])
    # add date label (year-calendar month e.g. 2002-01)
    year = np.floor(Ylms.time[i]).astype(np.int64)
    month = np.int64((Ylms.month[i]-1) % 12) + 1
    time_text.set_text(u'{0:4d}\u2013{1:02d}'.format(year,month))

# set animation
anim = animation.FuncAnimation(fig, animate_frames, frames=nt)
%matplotlib inline
HTML(anim.to_jshtml())